In [20]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torchmetrics.regression import R2Score


In [21]:
DATA_PATH = './Data/ETF_DATA/train.csv'

data_df = pd.read_csv(DATA_PATH, index_col = 0)
data_df

,market_price_1,market_price_2,market_price_3,market_price_4,market_price_5,market_price_6,market_price_7,market_price_8,market_price_9,market_price_10,...,closing_price_1,closing_price_2,closing_price_3,closing_price_4,closing_price_5,closing_price_6,closing_price_7,closing_price_8,closing_price_9,closing_price_10
0,"32,650","32,700","32,430","32,325","31,775","31,825","31,810","31,700","31,210","32,040",...,"32,735","32,240","32,150","31,805","32,060","31,285","31,690","31,580","31,940","31,705"
1,"32,700","32,430","32,325","31,775","31,825","31,810","31,700","31,210","32,040","31,835",...,"32,240","32,150","31,805","32,060","31,285","31,690","31,580","31,940","31,705","31,780"
2,"32,430","32,325","31,775","31,825","31,810","31,700","31,210","32,040","31,835","31,955",...,"32,150","31,805","32,060","31,285","31,690","31,580","31,940","31,705","31,780","32,175"
3,"32,325","31,775","31,825","31,810","31,700","31,210","32,040","31,835","31,955","32,120",...,"31,805","32,060","31,285","31,690","31,580","31,940","31,705","31,780","32,175","32,285"
4,"31,775","31,825","31,810","31,700","31,210","32,040","31,835","31,955","32,120","32,010",...,"32,060","31,285","31,690","31,580","31,940","31,705","31,780","32,175","32,285","32,170"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151496,"9,560","9,535","9,600","9,830","9,810","9,905","9,745","9,635","9,675","9,690",...,"9,520","9,545","9,810","9,835","9,745","9,665","9,620","9,690","9,655","9,605"
151497,"9,535","9,600","9,830","9,810","9,905","9,745","9,635","9,675","9,690","9,655",...,"9,545","9,810","9,835","9,745","9,665","9,620","9,690","9,655","9,605","9,545"
151498,"9,600","9,830","9,810","9,905","9,745","9,635","9,675","9,690","9,655","9,510",...,"9,810","9,835","9,745","9,665","9,620","9,690","9,655","9,605","9,545","9,585"
151499,"9,830","9,810","9,905","9,745","9,635","9,675","9,690","9,655","9,510","9,580",...,"9,835","9,745","9,665","9,620","9,690","9,655","9,605","9,545","9,585","9,660"


In [22]:
data_df.columns

Index(['market_price_1', 'market_price_2', 'market_price_3', 'market_price_4',
       'market_price_5', 'market_price_6', 'market_price_7', 'market_price_8',
       'market_price_9', 'market_price_10', 'high_price_1', 'high_price_2',
       'high_price_3', 'high_price_4', 'high_price_5', 'high_price_6',
       'high_price_7', 'high_price_8', 'high_price_9', 'high_price_10',
       'low_price_1', 'low_price_2', 'low_price_3', 'low_price_4',
       'low_price_5', 'low_price_6', 'low_price_7', 'low_price_8',
       'low_price_9', 'low_price_10', 'volume_1', 'volume_2', 'volume_3',
       'volume_4', 'volume_5', 'volume_6', 'volume_7', 'volume_8', 'volume_9',
       'volume_10', 'closing_price_1', 'closing_price_2', 'closing_price_3',
       'closing_price_4', 'closing_price_5', 'closing_price_6',
       'closing_price_7', 'closing_price_8', 'closing_price_9',
       'closing_price_10'],
      dtype='object')

In [23]:
data_df.drop(['market_price_10', 'high_price_10', 'low_price_10', 'volume_10'], axis = 1, inplace = True)

In [24]:
data_df.dropna(inplace = True, axis = 0)

In [25]:
len(data_df.columns)

46

In [26]:
data_df

,market_price_1,market_price_2,market_price_3,market_price_4,market_price_5,market_price_6,market_price_7,market_price_8,market_price_9,high_price_1,...,closing_price_1,closing_price_2,closing_price_3,closing_price_4,closing_price_5,closing_price_6,closing_price_7,closing_price_8,closing_price_9,closing_price_10
0,"32,650","32,700","32,430","32,325","31,775","31,825","31,810","31,700","31,210","32,910",...,"32,735","32,240","32,150","31,805","32,060","31,285","31,690","31,580","31,940","31,705"
1,"32,700","32,430","32,325","31,775","31,825","31,810","31,700","31,210","32,040","32,995",...,"32,240","32,150","31,805","32,060","31,285","31,690","31,580","31,940","31,705","31,780"
2,"32,430","32,325","31,775","31,825","31,810","31,700","31,210","32,040","31,835","32,440",...,"32,150","31,805","32,060","31,285","31,690","31,580","31,940","31,705","31,780","32,175"
3,"32,325","31,775","31,825","31,810","31,700","31,210","32,040","31,835","31,955","32,445",...,"31,805","32,060","31,285","31,690","31,580","31,940","31,705","31,780","32,175","32,285"
4,"31,775","31,825","31,810","31,700","31,210","32,040","31,835","31,955","32,120","31,905",...,"32,060","31,285","31,690","31,580","31,940","31,705","31,780","32,175","32,285","32,170"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151496,"9,560","9,535","9,600","9,830","9,810","9,905","9,745","9,635","9,675","9,630",...,"9,520","9,545","9,810","9,835","9,745","9,665","9,620","9,690","9,655","9,605"
151497,"9,535","9,600","9,830","9,810","9,905","9,745","9,635","9,675","9,690","9,535",...,"9,545","9,810","9,835","9,745","9,665","9,620","9,690","9,655","9,605","9,545"
151498,"9,600","9,830","9,810","9,905","9,745","9,635","9,675","9,690","9,655","9,645",...,"9,810","9,835","9,745","9,665","9,620","9,690","9,655","9,605","9,545","9,585"
151499,"9,830","9,810","9,905","9,745","9,635","9,675","9,690","9,655","9,510","9,835",...,"9,835","9,745","9,665","9,620","9,690","9,655","9,605","9,545","9,585","9,660"


In [27]:
features = data_df[data_df.columns[:-1]]
target = data_df[data_df.columns[-1:]]

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    random_state = 1,
                                                    test_size = 0.1)

X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train,
                                                  random_state = 1,
                                                  test_size = 0.1)

In [28]:
y_train

,closing_price_10
66565,"10,240"
137623,"11,560"
115806,"16,750"
121920,"17,445"
66264,"13,055"
...,...
143561,"6,375"
114561,"16,355"
37174,"10,140"
56419,"8,925"


In [29]:
X_train

,market_price_1,market_price_2,market_price_3,market_price_4,market_price_5,market_price_6,market_price_7,market_price_8,market_price_9,high_price_1,...,volume_9,closing_price_1,closing_price_2,closing_price_3,closing_price_4,closing_price_5,closing_price_6,closing_price_7,closing_price_8,closing_price_9
66565,"10,665","10,570","10,610","10,645","10,630","10,585","10,520","10,470","10,420","10,665",...,"485,711","10,600","10,645","10,630","10,585","10,580","10,460","10,420","10,380","10,315"
137623,"11,305","11,430","11,420","11,415","11,500","11,605","11,620","11,635","11,610","11,415",...,16,"11,465","11,425","11,395","11,575","11,620","11,540","11,605","11,610","11,570"
115806,"15,295","15,400","15,155","16,045","16,260","16,915","17,640","17,085","16,965","15,400",...,13,"15,000","15,430","15,945","16,295","17,530","17,045","16,820","17,120","16,750"
121920,"17,870","17,915","17,845","18,015","17,945","17,880","17,870","17,685","17,685","17,870",...,"2,077","17,845","17,845","17,945","17,905","17,880","17,715","17,690","17,650","17,705"
66264,"13,150","13,155","13,285","13,275","13,230","13,225","13,155","13,200","13,290","13,150",...,28,"13,255","13,275","13,265","13,235","13,005","12,975","13,005","12,900","12,945"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143561,"6,290","6,355","6,300","6,300","6,280","6,355","6,355","6,370","6,360","6,360",...,"6,117","6,275","6,245","6,250","6,340","6,395","6,320","6,340","6,405","6,385"
114561,"16,380","16,300","16,130","16,175","16,150","16,165","16,175","16,040","16,160","16,380",...,1,"16,300","16,130","16,175","16,150","16,165","16,175","16,040","16,160","16,170"
37174,"9,830","9,915","10,055","10,170","10,015","10,125","10,040","10,085","10,170","9,915",...,"6,895","10,095","10,020","10,015","10,000","10,040","10,115","10,135","10,175","10,160"
56419,"8,715","8,750","8,870","8,815","8,905","8,875","8,890","8,745","8,750","8,780",...,200,"8,845","8,990","8,920","8,825","8,935","8,770","8,735","8,885","8,900"


In [30]:
class CustomDataset(Dataset):
    
    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = self.featureDF.shape[0]
        self.n_cols = self.featureDF.shape[1]
    
    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        return featureTS, targetTS

In [31]:
class ANNModel(nn.Module):

    def __init__(self, in_in, in_out, out_out, h_ins = [], h_outs = [], dropout_prob = 0):
        super().__init__()

        self.in_layer = nn.Linear(in_in, h_ins[0] if len(h_ins) else in_out)
        self.dropout = nn.Dropout(p = dropout_prob)

        self.hd_layers = nn.ModuleList()
        for idx in range(len(h_ins) - 1):
            self.hd_layers.append(nn.Linear(h_ins[idx], h_outs[idx + 1]))
            self.hd_layers.append(nn.Dropout(p = dropout_prob))
        
        self.out_layer = nn.Linear(h_outs[-1] if len(h_outs) else in_out, out_out)

    def forward(self, input_data):
        y = F.relu(self.in_layer(input_data))
        for linear in self.hd_layers:
            y = F.relu(linear(y))
        
        return self.out_layer(y)

In [32]:
def del_punctuation(dataframe):
    for name in dataframe.columns:
        dataframe[name] = dataframe[name].str.replace(',', '')

    return dataframe

In [33]:
X_train = del_punctuation(X_train)
X_val = del_punctuation(X_val)
X_test = del_punctuation(X_test)

y_train = del_punctuation(y_train)
y_val = del_punctuation(y_val)
y_test = del_punctuation(y_test)

In [34]:
X_train

,market_price_1,market_price_2,market_price_3,market_price_4,market_price_5,market_price_6,market_price_7,market_price_8,market_price_9,high_price_1,...,volume_9,closing_price_1,closing_price_2,closing_price_3,closing_price_4,closing_price_5,closing_price_6,closing_price_7,closing_price_8,closing_price_9
66565,10665,10570,10610,10645,10630,10585,10520,10470,10420,10665,...,485711,10600,10645,10630,10585,10580,10460,10420,10380,10315
137623,11305,11430,11420,11415,11500,11605,11620,11635,11610,11415,...,16,11465,11425,11395,11575,11620,11540,11605,11610,11570
115806,15295,15400,15155,16045,16260,16915,17640,17085,16965,15400,...,13,15000,15430,15945,16295,17530,17045,16820,17120,16750
121920,17870,17915,17845,18015,17945,17880,17870,17685,17685,17870,...,2077,17845,17845,17945,17905,17880,17715,17690,17650,17705
66264,13150,13155,13285,13275,13230,13225,13155,13200,13290,13150,...,28,13255,13275,13265,13235,13005,12975,13005,12900,12945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143561,6290,6355,6300,6300,6280,6355,6355,6370,6360,6360,...,6117,6275,6245,6250,6340,6395,6320,6340,6405,6385
114561,16380,16300,16130,16175,16150,16165,16175,16040,16160,16380,...,1,16300,16130,16175,16150,16165,16175,16040,16160,16170
37174,9830,9915,10055,10170,10015,10125,10040,10085,10170,9915,...,6895,10095,10020,10015,10000,10040,10115,10135,10175,10160
56419,8715,8750,8870,8815,8905,8875,8890,8745,8750,8780,...,200,8845,8990,8920,8825,8935,8770,8735,8885,8900


In [35]:
X_train = X_train.astype('int')
X_val = X_val.astype('int')
X_test = X_test.astype('int')

y_train = y_train.astype('int')
y_val = y_val.astype('int')
y_test = y_test.astype('int')

In [36]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15149 entries, 55576 to 94070
Data columns (total 45 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   market_price_1   15149 non-null  int32
 1   market_price_2   15149 non-null  int32
 2   market_price_3   15149 non-null  int32
 3   market_price_4   15149 non-null  int32
 4   market_price_5   15149 non-null  int32
 5   market_price_6   15149 non-null  int32
 6   market_price_7   15149 non-null  int32
 7   market_price_8   15149 non-null  int32
 8   market_price_9   15149 non-null  int32
 9   high_price_1     15149 non-null  int32
 10  high_price_2     15149 non-null  int32
 11  high_price_3     15149 non-null  int32
 12  high_price_4     15149 non-null  int32
 13  high_price_5     15149 non-null  int32
 14  high_price_6     15149 non-null  int32
 15  high_price_7     15149 non-null  int32
 16  high_price_8     15149 non-null  int32
 17  high_price_9     15149 non-null  int32
 18  low_pri

In [37]:
# import pickle

rbscaler = RobustScaler().fit(X_train)
# with open('robust_scaler.pkl', 'wb') as f:
#     pickle.dump(rbscaler, f)

In [38]:
X_train_scaled = rbscaler.transform(X_train)
X_val_scaled = rbscaler.transform(X_val)
X_test_scaled = rbscaler.transform(X_test)

X_train_scaled

array([[-0.16751592, -0.18025478, -0.17620865, ..., -0.20140217,
        -0.20700637, -0.21551176],
       [-0.08598726, -0.07070064, -0.07315522, ..., -0.05035054,
        -0.05031847, -0.05594406],
       [ 0.42229299,  0.43503185,  0.40203562, ...,  0.61440408,
         0.65159236,  0.60267006],
       ...,
       [-0.27388535, -0.26369427, -0.24681934, ..., -0.23773104,
        -0.23312102, -0.23521933],
       [-0.41592357, -0.41210191, -0.3975827 , ..., -0.41618866,
        -0.39745223, -0.39542276],
       [-0.62101911, -0.64012739, -0.67366412, ..., -0.70427024,
        -0.71273885, -0.70438652]])

In [39]:
X_train = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_val = pd.DataFrame(X_val_scaled, columns = X_val.columns)
X_test = pd.DataFrame(X_test_scaled, columns = X_test.columns)

X_train

,market_price_1,market_price_2,market_price_3,market_price_4,market_price_5,market_price_6,market_price_7,market_price_8,market_price_9,high_price_1,...,volume_9,closing_price_1,closing_price_2,closing_price_3,closing_price_4,closing_price_5,closing_price_6,closing_price_7,closing_price_8,closing_price_9
0,-0.167516,-0.180255,-0.176209,-0.171429,-0.173858,-0.179341,-0.188094,-0.194937,-0.201643,-0.174189,...,16.126548,-0.176395,-0.170935,-0.173385,-0.179553,-0.179962,-0.196554,-0.201402,-0.207006,-0.215512
1,-0.085987,-0.070701,-0.073155,-0.073651,-0.063452,-0.050063,-0.048765,-0.047468,-0.051201,-0.078830,...,-0.133746,-0.065427,-0.071063,-0.075496,-0.053035,-0.047224,-0.058711,-0.050351,-0.050318,-0.055944
2,0.422293,0.435032,0.402036,0.514286,0.540609,0.622940,0.713743,0.642405,0.625790,0.427845,...,-0.133847,0.388069,0.441741,0.506718,0.550160,0.707084,0.643906,0.614404,0.651592,0.602670
3,0.750318,0.755414,0.744275,0.764444,0.754442,0.745247,0.742875,0.718354,0.716814,0.741894,...,-0.064747,0.753047,0.750960,0.762636,0.755911,0.751755,0.729419,0.725303,0.719108,0.724094
4,0.149045,0.149045,0.164122,0.162540,0.156091,0.155260,0.145662,0.150633,0.161188,0.141767,...,-0.133344,0.164208,0.165813,0.163788,0.159105,0.129547,0.124442,0.128107,0.114013,0.118881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122694,-0.724841,-0.717197,-0.724555,-0.723175,-0.725888,-0.715463,-0.715643,-0.713924,-0.714918,-0.721551,...,0.070506,-0.731238,-0.734315,-0.733845,-0.722045,-0.714103,-0.724952,-0.721479,-0.713376,-0.715194
122695,0.560510,0.549682,0.526081,0.530794,0.526650,0.527883,0.528182,0.510127,0.524020,0.552448,...,-0.134248,0.554843,0.531370,0.536148,0.531629,0.532865,0.532865,0.514978,0.529299,0.528926
122696,-0.273885,-0.263694,-0.246819,-0.231746,-0.251904,-0.237643,-0.248892,-0.243671,-0.233249,-0.269549,...,0.096552,-0.241180,-0.250960,-0.252079,-0.254313,-0.248883,-0.240587,-0.237731,-0.233121,-0.235219
122697,-0.415924,-0.412102,-0.397583,-0.403810,-0.392766,-0.396071,-0.394554,-0.413291,-0.412769,-0.413859,...,-0.127586,-0.401539,-0.382843,-0.392194,-0.404473,-0.389917,-0.412253,-0.416189,-0.397452,-0.395423


In [40]:
h_inouts = range(600, 40, -50)
dropout_prob = 0.25

ann_model = ANNModel(in_in = 45, in_out = 100, out_out = 1, h_ins = h_inouts, h_outs = h_inouts, dropout_prob = dropout_prob)
print(ann_model)

ANNModel(
  (in_layer): Linear(in_features=45, out_features=600, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (hd_layers): ModuleList(
    (0): Linear(in_features=600, out_features=550, bias=True)
    (1): Dropout(p=0.25, inplace=False)
    (2): Linear(in_features=550, out_features=500, bias=True)
    (3): Dropout(p=0.25, inplace=False)
    (4): Linear(in_features=500, out_features=450, bias=True)
    (5): Dropout(p=0.25, inplace=False)
    (6): Linear(in_features=450, out_features=400, bias=True)
    (7): Dropout(p=0.25, inplace=False)
    (8): Linear(in_features=400, out_features=350, bias=True)
    (9): Dropout(p=0.25, inplace=False)
    (10): Linear(in_features=350, out_features=300, bias=True)
    (11): Dropout(p=0.25, inplace=False)
    (12): Linear(in_features=300, out_features=250, bias=True)
    (13): Dropout(p=0.25, inplace=False)
    (14): Linear(in_features=250, out_features=200, bias=True)
    (15): Dropout(p=0.25, inplace=False)
    (16): Linear(in_features=20

In [41]:
EPOCH = 50
BATCH_SIZE = 32
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

In [42]:
trainDS = CustomDataset(X_train, y_train)

trainDL = DataLoader(trainDS, batch_size = BATCH_SIZE)

In [60]:
optimizer = optim.Adam(ann_model.parameters(), lr = LR)
regLoss = nn.MSELoss()
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 100, verbose = True)

c:\Users\KDP-2\anaconda3\envs\TORCH_CV_38\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [61]:
def testing(featureDF, targetDF, model):
    featureTS = torch.FloatTensor(featureDF.values).to(DEVICE)
    targetTS = torch.FloatTensor(targetDF.values).to(DEVICE)
    model.dropout = nn.Dropout(0)
    model.eval()
    with torch.no_grad():
        pre_val = model(featureTS)
        loss_val = regLoss(pre_val, targetTS)
        score_val = R2Score()(pre_val, targetTS)
    
    return loss_val, score_val, pre_val

In [62]:
SAVE_PATH = './saved_models/'
os.makedirs(SAVE_PATH, exist_ok = True)

def training(valDF, valtargetDF, model):

    BREAK_CNT_LOSS = 0
    BREAK_CNT_SCORE = 0
    LIMIT_VALUE = 10

    LOSS_HISTORY, SCORE_HISTORY = [[], []], [[], []]

    for epoch in range(EPOCH):
        SAVE_MODEL = os.path.join(SAVE_PATH, f'model_{epoch}.pth')
        SAVE_WEIGHT = os.path.join(SAVE_PATH, f'model_weights_{epoch}.pth')

        loss_total, score_total = 0, 0

        for featureTS, targetTS in trainDL:
            
            pre_y = model(featureTS)

            loss = regLoss(pre_y, targetTS)
            loss_total += loss.item()

            score = R2Score()(pre_y, targetTS)
            score_total += score.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        val_loss, val_r2, pre_val = testing(valDF, valtargetDF, model)

        LOSS_HISTORY[1].append(val_loss)
        SCORE_HISTORY[1].append(val_r2)

        LOSS_HISTORY[0].append(loss_total / len(trainDL))
        SCORE_HISTORY[0].append(score_total / len(trainDL))
        print(f"pre_val : {pre_val}\n y_val:{y_val}")
        print(f"[{epoch + 1}/{EPOCH}]\n - TRAIN LOSS : {LOSS_HISTORY[0][-1]} R2 : {SCORE_HISTORY[0][-1]}")
        print(f"- VAL LOSS : {LOSS_HISTORY[1][-1]} R2 : {SCORE_HISTORY[1][-1]}")

        scheduler.step(val_r2)

        if len(LOSS_HISTORY[1]) >= 2:
            if LOSS_HISTORY[1][-1] >= LOSS_HISTORY[1][-2]: BREAK_CNT_LOSS += 1
        
        if len(LOSS_HISTORY[1]) == 1:
            torch.save(model.state_dict(), SAVE_WEIGHT)
            torch.save(model, SAVE_MODEL)
        
        else:
            if LOSS_HISTORY[1][-1] < min(LOSS_HISTORY[1][:-1]):
                torch.save(model.state_dict(), SAVE_WEIGHT)
                torch.save(model, SAVE_MODEL)

        if BREAK_CNT_LOSS > LIMIT_VALUE:
            print(f"성능 및 손실 개선이 없어서 {epoch} EPOCH에 학습 중단")

    
    return LOSS_HISTORY, SCORE_HISTORY



In [63]:
loss, r2 = training(X_val, y_val, ann_model)

pre_val : tensor([[10284.9209],
        [13856.9229],
        [10388.7773],
        ...,
        [16023.9258],
        [ 7035.9819],
        [10936.5898]])
 y_val:        closing_price_10
34091              10560
29155              14405
122527             10585
137017             12965
11348              10715
...                  ...
86949               7270
107611             12595
115648             16125
50244               6800
22221              11385

[13634 rows x 1 columns]
[1/1000]
 - TRAIN LOSS : 143517194.85267276 R2 : 0.7623834089785232
- VAL LOSS : 11346652.0 R2 : 0.9843507409095764


KeyboardInterrupt: 

In [47]:
class RNNModel(nn.Module):
    def __init__(self, hidden_dim, input_size, n_layers, dropout,
                 bidirectional, model_type):
        super().__init__()

        if model_type == 'rnn':
            self.model = nn.RNN(
                input_size = input_size,
                hidden_size = hidden_dim,
                num_layers = n_layers,
                bidirectional = bidirectional,
                dropout = dropout,
                batch_first = True
            )
        
        elif model_type == 'lstm':
            self.model = nn.LSTM(
                input_size = input_size,
                hidden_size = hidden_dim,
                num_layers = n_layers,
                bidirectional = bidirectional,
                dropout = dropout,
                batch_first = True
            )
        
        if bidirectional:
            self.linear = nn.Linear(hidden_dim * 2, 1)
        
        else:
            self.linear = nn.Linear(hidden_dim, 1)

        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        output, _ = self.model(inputs)
        last_output = self.dropout(output)
        logits = self.linear(last_output)
        
        return logits

In [52]:
input_size = 45
hidden_dim = 512
n_layers = 2

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [53]:
rnn_model = RNNModel(input_size = input_size, hidden_dim = hidden_dim,
                      n_layers = n_layers, dropout = 0.9, bidirectional = False,
                      model_type = 'lstm').to(DEVICE)

In [65]:
EPOCH = 1000
LR = 0.001

regLoss = nn.L1Loss()
optimizer = optim.Adam(rnn_model.parameters(), lr = LR)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 100, verbose = True)


c:\Users\KDP-2\anaconda3\envs\TORCH_CV_38\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [66]:
loss, r2 = training(X_val, y_val, rnn_model)

pre_val : tensor([[10405.4727],
        [14237.4932],
        [10508.9609],
        ...,
        [15874.3750],
        [ 6963.7900],
        [11385.8975]])
 y_val:        closing_price_10
34091              10560
29155              14405
122527             10585
137017             12965
11348              10715
...                  ...
86949               7270
107611             12595
115648             16125
50244               6800
22221              11385

[13634 rows x 1 columns]
[1/1000]
 - TRAIN LOSS : 308.01367537039664 R2 : 0.9980932957035
- VAL LOSS : 268.5990295410156 R2 : 0.9995244145393372
pre_val : tensor([[10550.0273],
        [14496.3496],
        [10561.9844],
        ...,
        [15822.9561],
        [ 6843.7690],
        [11361.7314]])
 y_val:        closing_price_10
34091              10560
29155              14405
122527             10585
137017             12965
11348              10715
...                  ...
86949               7270
107611             12595
115

KeyboardInterrupt: 